# Inheritance

## Class Attributes

In the work covered so far, we have used **instance-level data**, eg the data is passed
to the class on instantiation. Eg -   
`foo = MyClass("Some data")`

If you would like to ensure some values are passed to all instances of the class, then
you can use a **Class Attribute** to achieve this. Eg -   
```
def MyClass:
    CONSTANT_VAR = 1
    ...

```

To then use this class attribute in `__init__()` or a method, you need to reference it
with the class name like:

`MyClass.CONSTANT_VAR`.


## Class Methods

You can also define 'global' methods, methods that are common to every instance of that
class. There are some pros and cons discussed. The main con is that class methods cannot
access instance level data. 

To initiate a class method, use a decorator:

```
def MyClass:

    @classmethod
    def some_method(cls, arg):
        # Do something
        return cls(some_returnable_object)

```

Note instead of `self` we now refer to `cls`. This is because `self` refers to the
instance, not the class. 

To use the class method we need to:

`MyClass.some_method(arg)`

***

## Updating Class Attributes

If you define a class attribute, updating that value needs to be done with
**assignment to the class, not the instance**.

Let's see:


In [63]:
from datetime import datetime # for IsoDate class
import pandas as pd

In [2]:
class SomeClass:
    A_CLASS_ATTRIBUTE = 1

c1 = SomeClass()
c2 = SomeClass()

print(c1.A_CLASS_ATTRIBUTE)
print(c2.A_CLASS_ATTRIBUTE)

1
1


In [3]:
c1.A_CLASS_ATTRIBUTE = 2
print(c1.A_CLASS_ATTRIBUTE)
print(c2.A_CLASS_ATTRIBUTE)
print(SomeClass.A_CLASS_ATTRIBUTE)

2
1
1


The class attribute was updated locally in the instance c1 only. In fact, python created a new instance level method with the same name, `A_CLASS_ATTRIBUTE`. Now lets update the
class attribute within the class itself and see the result on instances:

In [4]:
SomeClass.A_CLASS_ATTRIBUTE = 3
print(c1.A_CLASS_ATTRIBUTE)
print(c2.A_CLASS_ATTRIBUTE)
print(SomeClass.A_CLASS_ATTRIBUTE)

2
3
3


See that the instance c1 still returns 2, but the value 3 has propagated to c2. Looks
like a scoping inheritance rule.

Applying the class method decoartor in the context of date cleaning:

In [5]:
class IsoDate:
    """
    Uses class decorators for flexibility. Can handle string or datetime
    inputs. Returns integers in ISO order.
    """
    def __init__(self, y, m, d):
      self.y, self.m, self.d = y, m, d
      
    @classmethod
    def from_int_str(cls, strdate):
      """
      Handles string inputs in format YYYY-MM-DD.
      Flexible to seperators '-' or '/'
      """
      if "-" in strdate:
        y, m, d = map(int, strdate.split("-"))
      elif "/" in strdate:
        y,m,d = map(int, strdate.split("/"))
      return cls(y, m, d)
      
    @classmethod
    def from_dt(cls, dt):
      """
      Handles datetime inputs
      """
      y, m, d = dt.year, dt.month, dt.day
      return cls(y, m, d)


In [6]:
# investigate flexibility of class. First with strings:
id = IsoDate.from_int_str("2023-02-12")
print(f"The date returned is {id.y}-{id.m}-{id.d}")

The date returned is 2023-2-12


In [7]:
# different str format
id1 = IsoDate.from_int_str("2023/02/12")
print(f"The date returned is {id1.y}-{id1.m}-{id1.d}")

The date returned is 2023-2-12


In [8]:
id2 = IsoDate.from_dt(datetime.today())
print(f"today is {id2.y}-{id2.m}-{id2.d}")

today is 2023-2-12


***

## Class Inheritance

A good approach when reusing code without needing to rewrite it. Perhaps
you wish to riff on an existing class template without rewriting it. Using
class inheritance allows passing all of a classes' parts to a child class
for further modification. Let's take a look: 

In [25]:
class RaisePower:
    """
    Raise some integer to a power.
    """
    def __init__(self, base, exponent):
        self.b, self.exp = base, exponent

    def raise_pow(self):
        if self.exp > 0:
            self.out = self.b ** self.exp
            return self.out
        else:
            raise ValueError("exponent should be a positive numeric.")


In [26]:
bar = RaisePower(2, 3)
bar.raise_pow()

8

In [27]:
# Update the template with some pretty printing
class PrettyPrint(RaisePower):
    """
    Modifies RaisePow to include some pretty printing.
    """
    def do_print(self):
        print(f"Your answer is {self.out}")

In [30]:
foobar = PrettyPrint(3, 3)
print(foobar.raise_pow())
foobar.do_print()

27
Your answer is 27


In [32]:
# let's check some of these class instances to see what's inherited
print(isinstance(bar, RaisePower))
print(isinstance(bar, PrettyPrint))
print(isinstance(foobar, PrettyPrint))
print(isinstance(foobar, PrettyPrint))

True
False
True
True


You may refer to `foobar` as a sub-class or child class.

The subclass may have a constructor of its own, let's see how this is done.

In [44]:
class ModuloPower(RaisePower):
    def __init__(self, base, exponent, modulus):
        RaisePower.__init__(self, base, exponent)
        RaisePower.raise_pow(self)
        self.modulus = modulus
    def modulo_power(self):
        """
        Executes modulo on the output of RaisePower
        """
        self.out = self.out % self.modulus
        return self.out

In [47]:
fizz = ModuloPower(base=2, exponent=4, modulus=7)
fizz.modulo_power()
print(fizz.out)

2


***

## Subclass of a Pandas DF

Let's say we want to include a source column for every pd DF we work with.

In [89]:
class SourcedDF(pd.DataFrame):
    def __init__(self, *args, **kwargs):
        pd.DataFrame.__init__(self, *args, **kwargs)
        # use of args kwargs to copy over all pd parts without explicitly 
        # ensuring subclass signature is exact
        self.source = "TOP SECRET"

    def update_df(self):
        tmp = self.copy()
        tmp["source"] = self.source
        return tmp

In [91]:
buzz = SourcedDF({"A": ["a", "b", "c"]})
print(buzz)
print(buzz.source)
buzz.update_df()

   A
0  a
1  b
2  c
TOP SECRET


,A,source
0,a,TOP SECRET
1,b,TOP SECRET
2,c,TOP SECRET
